# Mezcla binaria (NP's y Mesógenos)

In [ ]:
#-----Importa los paquetenes necesartios para correr la simulación

from __future__ import division
import hoomd
import hoomd.md

#-----Define algunas variables relevantes

box         = 12.5;
replicates  = 10;
np_diameter = 5.0;
press       = 3.5;
t0          = ; 
t1          = ;

#-------Crea y define la celda unitaria

hoomd.context.initialize("");
uc = hoomd.lattice.unitcell(N = 1,
                            a1 = [box, 0, 0],
                            a2 = [0, box, 0],
                            a3 = [0, 0, box],
                            dimensions = 3,
                            position = [[0,0,0]],
                            type_name = ['M'],
                            mass = [1.0],
                            moment_inertia = [[0,
                                               1/12*1.0*8**2,
                                               1/12*1.0*8**2]],
                            orientation = [[1, 0, 0, 0]]);

#-----De la lattice obten un snapshot e inicializa el sistema desde ahí.

snap = uc.get_snapshot()
snap.replicate(replicates,replicates,replicates)
system = hoomd.init.read_snapshot(snap)

#-----Añade a las nanoparticulas.

system.particles.types.add('NP')
np_0 = system.particles[0]
np_1 = system.particles[222]
np_2 = system.particles[444]
np_3 = system.particles[666]

#-----Especifica el tipo, diametro, momento de inercia y "masa" de las NP's.

np_0.type = 'NP'
np_1.type = 'NP'
np_2.type = 'NP'
np_3.type = 'NP'

np_0.diameter = np_diameter
np_1.diameter = np_diameter
np_2.diameter = np_diameter
np_3.diameter = np_diameter


np_0.moment_inertia = [0,0,0]
np_1.moment_inertia = [0,0,0]
np_2.moment_inertia = [0,0,0]
np_3.moment_inertia = [0,0,0]

#-----

np_0.mass = 5.0
np_1.mass = 5.0
np_2.mass = 5.0
np_3.mass = 5.0

#-----Añade las cadenas de los mesógenos

system.particles.types.add('A');

#-----Define cada mesógeno en el marco de referencia local del cuerpo.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Crea los mesógenos.

rigid.create_bodies();

#-----Define la energía potencial.

nl = hoomd.md.nlist.cell()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Declara las interacciones entre elementos de la mezcla.

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0)
lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

lj.pair_coeff.set('NP','M', epsilon = 1.0, sigma = 3.0)
lj.pair_coeff.set('NP','A', epsilon = 1.0, sigma = 3.0)

#-----Selecciona un integrador estándar.

hoomd.md.integrate.mode_standard(dt=0.005);

#------Define dos grupos y efectua su unión.

nanoparticles = hoomd.group.type(name='Nano_Particles', type='NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#----Integra usando un NPT.

temp = hoomd.variant.linear_interp(points = [(----,t0), (----, t1)]);

npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = temp, tau = 0.5, tauP = 1.0, P = press);
npt.randomize_velocities(seed=42)

#-----Guarda los datos en archivos .log y .gsd

log_file = "NP_T_" + str(t1) + "_P_" + str(press) + "_ramp.log" ;
gsd_file = "NP_T_" + str(t1) + "_P_" + str(press) + "_ramp.gsd" ;
NPmeso_gsd_file = "NP_T_CM_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd" ;

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 

meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 

NPmeso_gsd = hoomd.dump.gsd(NPmeso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 

In [ ]:
hoomd.run(1e5)